# Парсинг МАС
#### Анастасия Костяницына, Александра Дахина 
28 декабря 2018

In [1]:
from bs4 import BeautifulSoup
from bs4 import Tag
import re
from string import punctuation, digits
from collections import defaultdict
import os
import pandas as pd
import numpy as np
from copy import deepcopy
import copy

In [2]:
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)

Пример структуры для грам. значений:

<superEntry>
    <metalemma></metalemma>
    <entry>
        <form>
            + <orth></orth>
            + <usg type="time"> temporal, historical era (‘archaic’, ‘old’, etc.) </usg>
            + <usg type="reg"> register = книжн, разг, </usg>
            + <usg type="style"> style (figurative, literal, etc.) + ирон., негативн., положит. </usg>
            + <usg type="dom"> domain, сфера деятельности</usg>
        </form>
        <gramGrp>
            + <pos> ч.речи </pos>
            + <gender>род</gender>
            + <num>число</num>"
            +<per>лицо</per>
            + <mood>наклонение</mood>
            + <asp> вид (совершенный/несовершенный) </asp>
            + <transit> переходный/непереходный </transit>
            <refl> возвратность </refl>
            + <pron_type> личное/неличное </pron_type>
        </gramGrp>
        <form type="inflected">
            + <case> ...</case>
            + <num>число</num>"
            <tns>...</tns>
            +  <orth  extent="part"> здесь изменяемая часть, например, если в словаре написано "мать, -ери", то в этот тэг пишем -ери и указываем extent="part", если в словаре написано "мать, матери", то сюда пишем "матери" и extent не указываем  </orth>
            + <orth  > если указано несколько форм, можно указать несколько orth, например для словарного входа "take; took, taken" можно написать сюда took и taken</orth>
        </form>
    </entry>
</superEntry>

## Функции:

In [3]:
def add_lemma(cur_soup, unique_entry):
    
    '''
    Добавляет лемму в шаблон
    '''

    cur_soup.metalemma.string = unique_entry
    cur_soup.orth.string = unique_entry
    cur_soup.orth['main'] = "True"

In [4]:
def add_pos(entr_soup, entry, data):
    
    '''
    Добавляет часть речи в шаблон
    '''

    ps = data.loc[data['mas'] == entry[0]]['_PoS'].tolist()[0]
    if ps != []:
        entr_soup.gramgrp.append(cur_soup.new_tag('pos'))
        entr_soup.gramgrp.pos.string = ps

In [5]:
def new_tag(parent, child, entr_soup, val, name='type', attr=False):
    
    '''
    Создает новый тег по заданным пар-ам
    '''
    
    new = entr_soup.new_tag(child)
    if attr:
        new[name] = attr
    new.string = val
    parent.append(new)

In [6]:
def searching(entr_soup, data, item, name, index=False):
    
    '''
    Ищет в столбце информацию, которая заключена в тегах b, u, i
    Ищет окончания слов (например, -а)
    
    b - вариант написания
    i - род, число, стиль, часть речи, вид, падеж
    u - формы слова 
    
    parent - то, куда потом нужно будет вставлять информацию, найденную в тегах
    '''
    
    gr = data.loc[data['mas'] == item][name].tolist()[0]
    if gr == '':
        return None, None

    if index is False:
        parent_f = entr_soup.find("form")
        parent_fi = entr_soup.find("form", type="inflected")
        parent_gr = entr_soup.find("gramgrp")
        parent = (parent_f, parent_fi, parent_gr)
        adding_soup = entr_soup.entry
    
    else:
        parent_sns = entr_soup.find("sense", n=str(index+1))
        parent_gr = parent_sns.find("gramgrp")
        parent = (parent_sns, parent_gr)
        adding_soup = parent_sns
        
    b = re.findall('\[b\](.*?)\[/b\]', gr)
    i = re.findall('\[i\](.*?)\[/i\]', gr)
    u = re.findall('\[u\](.*?)\[/u\]', gr)
    non = re.findall('(\-[a-zA-Zа-яА-Я]+?)\\b', gr)
    
    together = [b, i, u, non]
    
    if together == [[], [], [], []]:
        new_tag(adding_soup, 'note', entr_soup, gr)
    
    return parent, together

In [37]:
def in_tags(log, entr_soup, data, entry, tags, parent_fi, parent_f, parent_gr, adding_soup, index=False):
    
    '''
    Вставляет найденную информацию в тегах таблицы в нужное место шаблона
    '''

    global punctuation, dom_values, gender_values, num_values, case_values, reg_values, time_values, per_values, style_values, asp_values, transit_values, mood_values, pron_type_values 
  
    b, i, u, non = tags
    
    for val in non:
        if val not in log['orth_part']:
            new_tag(parent_fi, 'orth', entr_soup, val, name='extent', attr='part')
            log['orth_part'].add(val)
        
    
    for val in b: 
        val = val.strip(''.join(punctuation))
        if val not in log['orth']:
            new_tag(parent_f, 'orth', entr_soup, val)
            log['orth'].add(val)
    
    for val in u:
        if len(val) >= 2:
            if val[:2] == data.loc[data['mas'] == entry[0]]['Lemma_'].tolist()[0][:2]:
                if val not in log['orth']:
                    new_tag(parent_fi, 'orth', entr_soup, val)
                    log['orth'].add(val)
            else:
                if '-' not in val:
                    val = '-' + val
                if val not in log['orth_part']:
                    new_tag(parent_fi, 'orth', entr_soup, val, name='extent', attr='part')
                    log['orth_part'].add(val)
    
    for val in i:

        val = val.lower()
        gender = re.findall(gender_values, val)
        num = re.findall(num_values, val)
        reg = re.findall(reg_values, val)
        time = re.findall(time_values, val)
        style = re.findall(style_values, val)
        dom = re.findall(dom_values, val)
        asp = re.findall(asp_values, val)
        transit = re.findall(transit_values, val)
        mood = re.findall(mood_values, val)
        pron_type = re.findall(pron_type_values, val)
        per = re.findall(per_values, val)
        
        for g in gender: new_tag(parent_gr, 'gender', entr_soup, g)
        for g in num: new_tag(parent_gr, 'num', entr_soup, g)
        for g in reg: new_tag(parent_f, 'usg', entr_soup, g, attr="reg")
        for g in time: new_tag(parent_f, 'usg', entr_soup, g, attr="time")
        for g in style: new_tag(parent_f, 'usg', entr_soup, g, attr="style")
        for g in dom: new_tag(parent_f, 'usg', entr_soup, g, attr="dom")
        for g in asp: new_tag(parent_gr, 'asp', entr_soup, g)
        for g in transit: new_tag(parent_gr, 'transit', entr_soup, g)
        for g in mood: new_tag(parent_gr, 'mood', entr_soup, g)
        for g in pron_type: new_tag(parent_gr, 'pron_type', entr_soup, g)
        for g in per: new_tag(parent_gr, 'per', entr_soup, g)
            
        together = [gender, num, reg, time, style, dom, asp, transit, mood, pron_type, per]
            
        if index is False:
            case = re.findall(case_values, val)
            for g in case: new_tag(parent_fi, 'case', entr_soup, g)
            together.append(case)

        if together == [[] for _ in range(len(together))]:
            new_tag(adding_soup, 'note', entr_soup, val)
        

In [8]:
def add_gram(entr_soup, entry, data, name):
    '''
    Добавляет информацию из столбика (GrAll)
    '''

    log = defaultdict(set)
    parent, tags = searching(entr_soup, data, entry[0], name)
    
    if (parent, tags) != (None, None):
        parent_f, parent_fi, parent_gr = parent
        in_tags(log, entr_soup, data, entry, tags, parent_fi, parent_f, parent_gr, entr_soup, index=False)
    

In [9]:
def add_gram2_3(entr_soup, entry, data, name):
    
    '''
    Добавляет информацию из столбика (Gr2) или (Gr3)
    '''
    
    for index, item in enumerate(entry):

        log = defaultdict(set)
        parent, tags = searching(entr_soup, data, item, name, index=index)
        if (parent, tags) != (None, None):
            parent_sns, parent_gr = parent
            in_tags(log, entr_soup, data, entry, tags, parent_sns, parent_sns, parent_gr, parent_sns, index=index)

In [10]:
def create_entry():
    
    '''
    Если слово может иметь несколько частей речи, они оформляются как hom.
    Функуия сосздает дополнительные entry
    '''
    
    ex = '<entry><form><orth></orth></form><gramGrp></gramGrp><form type="inflected"></form><sense></sense></entry>'
    soup = BeautifulSoup(ex, 'lxml')
    soup.entry['type'] = 'hom'
    return soup

In [43]:
def add_ex(entr_soup, entry_data, data):
    
    '''
    Вставляет в шаблон примеры
    '''
    
    for indx, entr in enumerate(entry_data):
        cur_soup = BeautifulSoup(copy.copy(ex_), 'lxml')
        tag_ex = cur_soup.find("cit")
        
        cl = data.loc[data['mas'] == entr]['(Ex)'].tolist()[0]
    
        ex_list = cl.split('[i]')
        for ind, el in enumerate(ex_list):
            el = re.sub(' \? ', ' ', el)
            el = re.sub('^\? ', ' ', el)
            el = re.sub('^ ', '', el)
            el = re.sub('[<>\#\^\*]', '', el)
            el = re.sub(' +', ' ', el)

            ex_list[ind] = el
    
    
        for q in ex_list:
            if q != '':
                q_l = q.split('[/i]')
                if len(q_l) != 1:
                    qut = BeautifulSoup(copy.copy(qu), 'lxml')
                    tag_q1 = qut.find("quote")
                    q_l[0] = q_l[0].strip()
                    tag_q1.string = q_l[0]

                    q_bibl = q_l[1].split(',')

                    tag_q2 = qut.find("author")
                    q_bibl[0] = q_bibl[0].strip()
                    tag_q2.string = q_bibl[0]

                    tit = ','.join(q_bibl[1:])
                    tag_q3 = qut.find("title")
                    tit = tit.strip()
                    tag_q3.string = tit

                    cur_soup.cit.append(qut.quote)
                    cur_soup.cit.append(qut.bibl)
                
        cur_soup.sense['n'] = str(indx+1)
    
        entr_soup.entry.append(cur_soup.sense)

In [39]:
def add_ex_in(entr_soup, entry_data):
    
    '''
    Вставляет в шаблон с подзначениями примеры
    '''
    
    tag_ex = entr_soup.find("cit")


    ex_list = entry_data.split('[i]')
    for ind, el in enumerate(ex_list):
        el = re.sub(' \? ', ' ', el)
        el = re.sub('^\? ', ' ', el)
        el = re.sub('^ ', '', el)
        el = re.sub('[<>\#\^\*]', '', el)
        el = re.sub(' +', ' ', el)

        ex_list[ind] = el


    for q in ex_list:
        if q != '':
            q_l = q.split('[/i]')
            if len(q_l) != 1:
                qut = BeautifulSoup(copy.copy(qu), 'lxml')
                tag_q1 = qut.find("quote")
                q_l[0] = q_l[0].strip()
                tag_q1.string = q_l[0]

                #print(q_l)
                q_bibl = q_l[1].split(',')

                tag_q2 = qut.find("author")
                q_bibl[0] = q_bibl[0].strip()
                tag_q2.string = q_bibl[0]

                tit = ','.join(q_bibl[1:])
                tag_q3 = qut.find("title")
                tit = tit.strip()
                tag_q3.string = tit

                entr_soup.cit.append(qut.quote)
                entr_soup.cit.append(qut.bibl)

In [62]:
def add_def(entr_soup, entry_data, data):
    
    '''
    Вставляет в шаблон значение
    '''
    
    alf = 'qwertyuioplkjhgfdsazxcvbnm'
    
    for indx, entr in enumerate(entry_data):
    
        cl = data.loc[data['mas'] == entr]['(Def)'].tolist()[0]
        
    
        cl = re.sub('\.\#\[[^i]', '.####[', cl)
        
        for link in entr_soup.find_all('sense'):
                s = link.get('n')
                if not s is None:
                    if s not in alf:
                        try:
                            s_i = int(s)
                            if int(s) == indx+1:
                                s_soup = link
                        
                        except ValueError:
                            print(s)
                        
        
        
        def_list = re.split('####', cl)
        if len(def_list) == 1:
            sm = re.findall('\[i\]см.\[/i\] \[u\].+?\[/u\]', def_list[0])
            if sm != []:
                sm_str = ' '.join(sm)
                sm_str = re.sub('\[/?.*?\]', '', sm_str)
                sm_str = re.sub(' \? ', ' ', sm_str)
                sm_str = re.sub('^\? ', ' ', sm_str)
                sm_str = re.sub('^ ', '', sm_str)
                sm_str = re.sub('[<>\#\^\*]', '', sm_str)
                sm_str = re.sub(' +', ' ', sm_str)
                s_soup.xr.append(sm_str)
            
            idim = re.findall('\[b?\].+?\[/b\]', def_list[0])
            if idim != []:
                idim_str = ' '.join(idim)
                idim_str = re.sub('\[/?.\]', '', idim_str)
                idim_str = re.sub(' \? ', ' ', idim_str)
                idim_str = re.sub('^\? ', ' ', idim_str)
                idim_str = re.sub('^ ', '', idim_str)
                idim_str = re.sub('[<>\#\^\*]', '', idim_str)
                idim_str = re.sub(' +', ' ', idim_str)
                s_soup.re.append(idim_str)
            

            el_cl = re.sub('\[b\].+?\[/b\]', ' ', def_list[0])
            el_cl = re.sub('\[i\]см.\[/i\] \[u\].+?\[/u\]', ' ', el_cl)
            
            if el_cl != '':
                
                if '[/i]' in el_cl:
                    add_ex_in(s_soup, el_cl)
                
                el_cl = re.sub('\[/?.\]', '', el_cl)
                el_cl = re.sub('[<>\#\^\*]', '', el_cl)
                el_cl = re.sub(' \?', ' ', el_cl)
                el_cl = re.sub(' +', ' ', el_cl)
            
            
            

            def_soup = BeautifulSoup(copy.copy(def_q), 'lxml')

            tag_def = def_soup.find("def_text")
            tag_def.string = el_cl



            tag_def1 = s_soup.find("def")
            
            tag_def1.append(tag_def)


        elif len(def_list) > 1:
            parse_indef(entr_soup, def_list, indx)
            
        

In [61]:
def parse_indef(entr_soup, def_list, indx):
    
    '''
    Вставляет в шаблон подзначения
    '''
    
    alf = 'qwertyuioplkjhgfdsazxcvbnm'
    
    alf = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's'
          't', 'u', 'v', 'w', 'x', 'y', 'z']
    alf_ind = 0
    
    for link in entr_soup.find_all('sense'):
        s = link.get('n')
        if not s is None:
            if s not in alf:
                try:
                    s_i = int(s)
                    if int(s) == indx+1:
                        s_soup = link

                except ValueError:
                    print(s)


    tag_def = s_soup.find("def")
    
    for ind, el in enumerate(def_list):
        
        soup = BeautifulSoup(copy.copy(ex_), 'lxml')
        
        
        sm = re.findall('\[i\]см.\[/i\] \[u\].+?\[/u\]', el)
        if sm != []:
            sm_str = ' '.join(sm)
            sm_str = re.sub('\[/?.\]', '', sm_str)
            sm_str = re.sub(' \? ', ' ', sm_str)
            sm_str = re.sub('^\? ', ' ', sm_str)
            sm_str = re.sub('^ ', '', sm_str)
            sm_str = re.sub('[<>\#\^\*]', '', sm_str)
            sm_str = re.sub(' +', ' ', sm_str)
            soup.xr.append(sm_str)
        
        idim = re.findall('\[b?\].+?\[/b\]', el)
        if idim != []:
            idim_str = ' '.join(idim)
            idim_str = re.sub('\[/?.*?\]', '', idim_str)
            idim_str = re.sub(' \? ', ' ', idim_str)
            idim_str = re.sub('^\? ', ' ', idim_str)
            idim_str = re.sub('^ ', '', idim_str)
            idim_str = re.sub('[<>\#\^\*]', '', idim_str)
            idim_str = re.sub(' +', ' ', idim_str)
            soup.re.append(idim_str)
        
        el_cl = re.sub('\[b\].+?\[/b\]', ' ', el)
        el_cl = re.sub('\[i\]см.\[/i\] \[u\].+?\[/u\]', ' ', el)
        
        if el_cl != '':
            if '[/i]' in el_cl:
                add_ex_in(soup, el_cl)
                
                
            indef_gram(el_cl, soup, '(GrAll)')
            el_cl = re.sub('\[/?.\]', '', el_cl)
            el_cl = re.sub(' \? ', ' ', el_cl)
            el_cl = re.sub('^\? ', ' ', el_cl)
            el_cl = re.sub('^ ', '', el_cl)
            el_cl = re.sub('[<>\#\^\*]', '', el_cl)
            el_cl = re.sub(' +', ' ', el_cl)
            
            def_soup = BeautifulSoup(copy.copy(def_q), 'lxml')

            tag_def2 = def_soup.find("def_text")
            tag_def2.string = el_cl

            tag_def1 = soup.find("def")
            tag_def1.append(tag_def2)

        try:
            soup.sense['n'] = alf[alf_ind]
            alf_ind+=1
            tag_def.append(soup.sense)
            
        except IndexError:
            #print(tag_def.prettify())
            break

In [17]:
def indef_gram(gr, entr_soup, name):

    '''
    бывает, что у подзначений есть стилистические характеристики. 
    '''
    global dom_values, gender_values, num_values, case_values, reg_values, time_values, style_values, asp_values, transit_values, mood_values, pron_type_values 
    
        
    b = re.findall('\[b\](.*?)\[/b\]', gr)
    i = re.findall('\[i\](.*?)\[/i\]', gr)
    u = re.findall('\[u\](.*?)\[/u\]', gr)
    non = re.findall('(\-[a-zA-Zа-яА-Я]+?)\\b', gr)
    parent_f = entr_soup.find("form")
    parent_fi = entr_soup.find("form", type="inflected")
    parent_gr = entr_soup.find("gramgrp")
    
    
    
    log = defaultdict(set)


    for val in i:
        val = val.lower()
        gender = re.findall(gender_values, val)
        num = re.findall(num_values, val)
        case = re.findall(case_values, val)
        reg = re.findall(reg_values, val)
        time = re.findall(time_values, val)
        style = re.findall(style_values, val)
        dom = re.findall(dom_values, val)
        asp = re.findall(asp_values, val)
        transit = re.findall(transit_values, val)
        mood = re.findall(mood_values, val)
        pron_type = re.findall(pron_type_values, val)
        
        for g in gender: new_tag(parent_gr, 'gender', entr_soup, g)
        for g in num: new_tag(parent_gr, 'num', entr_soup, g)
        for g in case: new_tag(parent_gr, 'case', entr_soup, g)
        for g in reg: new_tag(parent_gr, 'usg', entr_soup, g, attr="reg")
        for g in time: new_tag(parent_gr, 'usg', entr_soup, g, attr="time")
        for g in style: new_tag(parent_gr, 'usg', entr_soup, g, attr="style")
        for g in dom: new_tag(parent_gr, 'usg', entr_soup, g, attr="dom")
        for g in asp: new_tag(parent_gr, 'asp', entr_soup, g)
        for g in transit: new_tag(parent_gr, 'transit', entr_soup, g)
        for g in mood: new_tag(parent_gr, 'mood', entr_soup, g)
        for g in pron_type: new_tag(parent_gr, 'pron_type', entr_soup, g)
    
     

Регулярные выражения для поиски информации

In [12]:
dom_values = '\\b(Сад\.|с\.\-х|Спец\.|Архит\.|Авиа\.|Опт\.|астр\.|Мед\.|Зоол\.|этногр\.|ист\.|Физ\.|хим\.|Мат\.|Обл\.|Фин\.|Ж\.-д\.|Астр\.|Психол\.|Грамм\.|лит\.|Полигр\.|Тех\.|Дипл\.|Муз\.|Биол\.|Пед\.|Филос\.|Метеор\.|Бухг\.|Церк\.|Трад\.\-поэт\.|Лингв\.|Юр\.|Геол\.|Биохим\.|Анат\.|Охот\.|Кулин\.|Воен\.)'.lower()
gender_values = '\\b(ж\.|м\.|ср\.)'
num_values = '\\b(мн\.|ед\.)'
case_values = '\\b(им\.|род\.|вин\.|винительным|дат\.|предл\.|твор\.|творительным)'
reg_values = '\\b(Разг\.|Книжн\.|Прост\.|прост\.|книжн\.|разг\.|Груб\. прост\.|Прост\. пренебр\.)'.lower()
time_values = '\\b(устар\.)'
style_values = '\\b(ирон\.|высок\.|шутл\.-ирон\.)'
asp_values = '\\b(сов\.|несов\.)'
transit_values = '\\b(перех\.|неперех\.)'
mood_values = '\\b(повел\.)'
pron_type_values = '\\b(личн\.|неличн\.)'
per_values = '\\b([1-3] ?л\.)'

Шаблон вхождения

In [23]:
xml_n = '<superEntry><metalemma></metalemma><entry><form><orth></orth></form><gramGrp></gramGrp><form type="inflected"></form><sense></sense></entry></superEntry>'


In [18]:
def_q = '<def_text></def_text>'
idiom = '<re type="idiom"></re>'
ex_ = '<sense><gramGrp></gramGrp><def></def><cit type="example"></cit><re type="idiom"></re><xr></xr></sense>'
qu = '<quote></quote><bibl><author></author><title></title></bibl>'

## Данные

In [56]:
data = pd.read_csv('MAS/masA-J.csv',  sep=';', na_values= np.nan)
#data = pd.read_csv('/Users/Stoneberry/Desktop/Uni/Мастерская/MAS/masR_Ja.csv',  sep=';', na_values= np.nan)
data = data.fillna('')

c:\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
data.head()

,0001,mas,Lex,Entr,Lemma,(Gr),(GrAll),(n),(Gr2),Lemma_,_PoS,_n,арестовать,Unnamed: 13,(Def),(Gr3),(Ex)
0,2,масА0001A,а1,А1,а,"[i]нескл., ср.[/i]","[i]нескл., ср.[/i]",%n=0,,а,С,0,баловать,,Название первой буквы русского алфавита.,,
1,3,масА0001B,,,а,,,%%?,,а,,,бегать,,[b]От а до зет; от а до я[/b]#? от начала до к...,,
2,4,масА0002A,а2,А2,а,[i]союз.[/i],[i]союз.[/i],%n=I,[i]противительный.[/i],а,СОЮЗ,I,бежать,,,,
3,5,масА0002B,,,а,,[i]союз.[/i],%n=1,,а,СОЮЗ,1,беречь,,Употребляется для соединения противопоставляем...,,"[i]Он шутил, а я злобствовал.[/i] Пушкин, Выст..."
4,6,масА0002C,,,а,,[i]союз.[/i],%||,,а,СОЮЗ,1,беситься,,При отрицании служит противопоставлению одноро...,,"[i]Бывало грустно им, а скучно не бывало.[/i] ..."


Чтобы обеспечить простоту парсинга и сохранность структуры, групируем строки таблицы по леммам так, чтобы для каждой леммы отдельно были сгруппированы строки по части речи.

In [58]:
groups = data.groupby('Lemma_')['mas'].apply(list)
d = defaultdict(list)

# каждое слово может иметь несколько омографов, группируем по ним
for unique_entry in groups:
    lemma = data.loc[data['mas'] == unique_entry[0]]['Lemma_'].tolist()[0]
    new = data.loc[data['mas'].isin(unique_entry)]
    new_groups = new.groupby('Lemma')['mas'].apply(list)
    d[lemma] = [i for i in new_groups]

In [28]:
data.loc[data['Lemma_'] == 'раз']['_PoS']

369           COM
370           COM
371           COM
372           COM
373             С
374             С
375             С
376             С
377             С
378             С
379    ПРЕДИКАТИВ
380         НАРЕЧ
381          СОЮЗ
382          СОЮЗ
Name: _PoS, dtype: object

In [29]:
d['раз']

[['масР0198A', 'масР0198B', 'масР0198C', 'масР0198D'],
 ['масР0199A',
  'масР0199B',
  'масР0199C',
  'масР0199D',
  'масР0199E',
  'масР0199F'],
 ['масР0200A'],
 ['масР0201A'],
 ['масР0202A', 'масР0202B']]

Все распаршенные вхождения

In [63]:
all_ = [] 

Проходимся по таблице

In [64]:
for unique_entry in d:

    cur_soup = BeautifulSoup(copy.copy(xml_n), 'lxml')
    add_lemma(cur_soup, unique_entry)
    
    for index, entry in enumerate(d[unique_entry]):

        not_main = False

        if index == 0:
            cur_soup.entry['type'] = 'main' 
            entr_soup = cur_soup 
        else: 
            entr_soup = create_entry() # entr_soup - entry каждого значения
            entr_soup.orth.string = unique_entry

        add_pos(entr_soup, entry, data)
        add_gram(entr_soup, entry, data, '(GrAll)')
        add_ex(entr_soup, entry, data)
        add_def(entr_soup, entry, data)
        add_gram2_3(entr_soup, entry, data, '(Gr2)')
        add_gram2_3(entr_soup, entry, data, '(Gr3)')
        cur_soup.superentry.append(entr_soup.entry)

    all_.append(cur_soup)

st
st
st
st
st
st
st
st


In [65]:
all_[:2]

[<html><body><superentry><metalemma>а</metalemma><entry type="main"><form><orth main="True">а</orth></form><gramgrp><pos>С</pos><gender>ср.</gender></gramgrp><form type="inflected"></form><sense></sense><sense n="1"><gramgrp></gramgrp><def><def_text>Название первой буквы русского алфавита.</def_text></def><cit type="example"></cit><re type="idiom"></re><xr></xr></sense><sense n="2"><gramgrp></gramgrp><def><def_text> от начала до конца; всё целиком.</def_text></def><cit type="example"></cit><re type="idiom">От а до зет; от а до я</re><xr></xr></sense><sense n="3"><gramgrp></gramgrp><def><def_text></def_text></def><cit type="example"></cit><re type="idiom"></re><xr></xr><note>противительный.</note></sense><sense n="4"><gramgrp></gramgrp><def><def_text>Употребляется для соединения противопоставляемых предложений или отдельных членов предложения; соответствует по значению словам: но, зато, наоборот, напротив.</def_text></def><cit type="example"><quote>Он шутил, а я злобствовал.</quote><bib

In [66]:
f_1 = open('MAS/masA-J.txt', 'w', encoding='UTF-8')
f_1.close()

In [67]:
with open('MAS/masA-J.txt', 'a', encoding='utf-8') as g:
    for el in all_:
        g.write(str(el.superentry))
    

In [69]:
head = '<tei>\
 <file_desc>\
  <respstmt>\
   <name>\
    Костяницына Анастасия, Матвеева Александра\
   </name>\
  </respstmt>\
  <extent>\
    количество словарных входов: 150 454\
  </extent>\
  <sourcedesc>\
   <ref target="ссылка на словарь в интернете">\
     http://feb-web.ru/feb/mas/MAS-abc/default.asp\
   </ref>\
   <p>\
     This database was converted from the MAS Dictionary.\
    </p>\
  </sourcedesc>\
 </file_desc>\
 <front>\
  <head>\
   <title dict_id="">\
     Малый Академический Словарь, МАС\
   </title>\
   <volume n="" of="">\
   </volume>\
   <author>\
    РАН, Инcтитут лингвистических исследований\
   </author>\
   <publisher>\
   </publisher>\
   <editor>\
    А.П. Евгеньева\
   </editor>\
   <edition>\
    4-е издание, стереотипное\
   </edition>\
   <translator>\
   </translator>\
   <pubdate>\
    1999\
   </pubdate>\
   <isbn>\
   </isbn>\
   <language type="content">\
    <idno type="iso639-3">\
     rus\
    </idno>\
   </language>\
   <script type="rus">\
   </script>\
  </head>\
 </front>\
 <body>'

In [70]:
f_1 = open('MAS/mas_parsed.tei', 'w', encoding='UTF-8')
f_1.close()

In [73]:
files = ['MAS/masA-J.txt', 'MAS/masK-P.txt', 'MAS/masR_Ja.txt']

In [74]:
for fil in files:
    with open(fil, 'r', encoding='utf-8') as g:
        text = g.read()
        head += text

with open('MAS/mas_parsed.tei', 'a', encoding='utf-8') as gr:
    gr.write(head+'</body></tei>')